
**Install requirements**

In [0]:
!pip3 install 'torch==1.3.1'
!pip3 install 'torchvision==0.4.2'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'

**Import libraries**

In [0]:
import os
import logging
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.hub
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import alexnet

from PIL import Image
from tqdm import tqdm

from torch.hub import load_state_dict_from_url
from torch.autograd import Function
import copy

**Set Arguments**

In [0]:
DEVICE = 'cuda' 

NUM_CLASSES = 7 
NUM_DOMAINS = 2

#hyperparameters for AlexNet without dann
LR         = [1e-3, 2e-3, 5e-3, 1e-2, 1e-3, 2e-3, 5e-3, 1e-2]
BATCH_SIZE = [128,  128,  128,  128,  256,  256,  256,  256] 
NUM_EPOCHS = [15,   15,   15,   15,   15,   15,   15,   15]     
STEP_SIZE  = [10,   10,   10,   10,   10,   10,   10,   10] 

#hyperparameters for AlexNet with dann
LR_DANN         = [1e-3, 5e-3, 1e-2] 
BATCH_SIZE_DANN = [128, 256]  
NUM_EPOCHS_DANN = 15     
STEP_SIZE_DANN  = 10
ALPHA           = [1e-2, 5e-2, 1e-1]

MOMENTUM = 0.9      
WEIGHT_DECAY = 5e-5  
GAMMA = 0.1          
LOG_FREQUENCY = 10

**DANN**

In [0]:
__all__ = ['DANN', 'dann']


model_urls = {
    'dann': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}

class ReverseLayerF(Function):
    # Forwards identity
    # Sends backward reversed gradients
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

class DANN(nn.Module):

    def __init__(self, num_classes=1000):
        super(DANN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.Gy = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

        self.Gd = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )
    
    def forward(self, x, alpha=None):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        # If we pass alpha, we can assume we are training the discriminator
        if alpha is not None:
            # gradient reversal layer (backward gradients will be reversed)
            reverse_feature = ReverseLayerF.apply(x, alpha)
            discriminator_output = self.Gd(reverse_feature)
            return discriminator_output
        # If we don't pass alpha, we assume we are training with supervision
        else:
            # do something else
            class_outputs = self.Gy(x)
            return class_outputs

def dann(pretrained=False, progress=True, **kwargs):
    r"""AlexNet model architecture from the
    `"One weird trick..." <https://arxiv.org/abs/1404.5997>`_ paper.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    model = DANN(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['dann'], progress = progress)
        model.load_state_dict(state_dict, strict = False)
    return model

**Define Data Preprocessing**

In [0]:
normalize = transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
train_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      normalize,])

eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      normalize,])

**Prepare Dataset**

In [0]:
# Clone github repository with data
if not os.path.isdir('./Homework3-PACS'):
  !git clone https://github.com/MachineLearning2020/Homework3-PACS.git

DATA_DIR = 'Homework3-PACS/PACS'
DATA_DIR_SOURCE = 'Homework3-PACS/PACS/photo'
DATA_DIR_TARGET = 'Homework3-PACS/PACS/art_painting'

# Prepare Pytorch train/test Datasets for alexnet
train_dataset = torchvision.datasets.ImageFolder(DATA_DIR_SOURCE, transform = train_transform)
test_dataset  = torchvision.datasets.ImageFolder(DATA_DIR_TARGET, transform = eval_transform)

#Prepare Source and Target for dann
source_dataset = torchvision.datasets.ImageFolder(DATA_DIR_SOURCE, transform = train_transform)
target_dataset = torchvision.datasets.ImageFolder(DATA_DIR_TARGET, transform = train_transform)

#validation
DATA_DIR_VAL_C = 'Homework3-PACS/PACS/cartoon'
DATA_DIR_VAL_S = 'Homework3-PACS/PACS/sketch'

val_cartoon = torchvision.datasets.ImageFolder(DATA_DIR_VAL_C, transform = eval_transform)
val_sketch  = torchvision.datasets.ImageFolder(DATA_DIR_VAL_S, transform = eval_transform)

**Computation of accuracy**

In [0]:
def test_accuracy(net, dataloader, type_of_set):  
    # check accuracy on whole test set
    correct = 0
    total = 0
    net.train(False)  
    with torch.no_grad(): 
        for data in dataloader:
            images, labels = data
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = net(images)  # predictions
            _, predicted = torch.max(outputs.data, 1)  # predicted labels
            total += labels.size(0)
            correct += torch.sum(predicted == labels.data).data.item()  # compare with ground truth
    accuracy = 100 * correct / total
    print('Accuracy of the network on the %s set: %.3f %%' %(type_of_set, accuracy))
    net.train(True)
    return accuracy

**print losses**

In [0]:
def print_loss(loss, title):
  plt.plot(loss, 'b', label =  title) 
  plt.legend(loc = 'best')
  plt.xlabel('Epochs')
  plt.ylabel('Loss values')
  plt.grid()
  plt.show() 

**Print accuracy**

In [0]:
def print_accuracy(train_acc, val_acc, train_title, val_title):
  plt.plot(train_acc, 'r',label = train_title)
  plt.plot(val_acc,   'g', label = val_title)
  plt.legend(loc = 'best')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy values')
  plt.grid()
  plt.show()

**Test function**

In [0]:
def test(test_net, dataloader, dataset):
  net = test_net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
  net.train(False) # Set Network to evaluation mode

  running_corrects = 0
  for images, labels in tqdm(dataloader):
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    # Forward Pass
    outputs = net(images)

    # Get predictions
    _, preds = torch.max(outputs.data, 1)

    # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()

  # Calculate Accuracy
  accuracy = running_corrects / float(len(dataset))

  print('Test Accuracy: {}'.format(accuracy))
    

**Final Train and Test with DANN**

In [0]:
def final_train_and_test_dann(best_lr, best_batch, best_step, best_epoch, best_alpha):
  print('best hiperparameters: ')
  print('lr = {}' .format(best_lr))
  print('batch size = {}' .format(best_batch))
  print('step size = {}' .format(best_step))
  print('epoch = {}' .format(best_epoch))
  print('alpha = {}' .format(best_alpha))

  #define the net
  dann_net = dann(pretrained = True)
  dann_net.Gy[6] = nn.Linear(4096, NUM_CLASSES)
  dann_net.Gd[6] = nn.Linear(4096, NUM_DOMAINS)

  dann_net.Gd[1].weight.data = copy.deepcopy(dann_net.Gy[1].weight.data)
  dann_net.Gd[1].bias.data   = copy.deepcopy(dann_net.Gy[1].bias.data)

  dann_net.Gd[4].weight.data = copy.deepcopy(dann_net.Gy[4].weight.data)
  dann_net.Gd[4].bias.data   = copy.deepcopy(dann_net.Gy[4].bias.data)
  dann_net = dann_net.to(DEVICE)

  parameters_to_optimize = dann_net.parameters()
  optimizer = optim.SGD(parameters_to_optimize, lr = best_lr, momentum = MOMENTUM, weight_decay = WEIGHT_DECAY)
  scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = best_step, gamma = GAMMA)

  source_dataloader        = DataLoader(source_dataset, batch_size = best_batch, shuffle = True,  num_workers = 4, drop_last = True)
  target_dataloader        = DataLoader(target_dataset, batch_size = best_batch, shuffle = False, num_workers = 4)

  j = 0 
  current_step = 0

  cudnn.benchmark
  n_loss_print = len(source_dataloader)
  
  accuracies_Gy = np.empty(NUM_EPOCHS_DANN)

  losses_Gy = np.empty(NUM_EPOCHS_DANN)
  losses_Gd_source = np.empty(NUM_EPOCHS_DANN)
  losses_Gd_target = np.empty(NUM_EPOCHS_DANN)

  # Start iterating over the epochs
  for epoch in range(NUM_EPOCHS_DANN):
    print('Starting epoch {}/{}, LR = {}, batch size = {}, alpha = {}'.format(epoch+1, NUM_EPOCHS_DANN, scheduler.get_lr(), best_batch, best_alpha))
    target_dataloader_iterator = iter(target_dataloader)
    running_loss_Gy = 0.0
    running_loss_Gd_source = 0.0
    running_loss_Gd_target = 0.0
    # Iterate over the dataset
    for source_images, source_labels in source_dataloader:
      
      source_images = source_images.to(DEVICE)
      source_labels = source_labels.to(DEVICE)

      
      dann_net.train() # Sets module in training mode
      optimizer.zero_grad() # Zero-ing the gradients

      # First step
      outputs = dann_net.forward(source_images) # forward pass
      loss_Gy = criterion(outputs, source_labels)
      running_loss_Gy += loss_Gy.item()
      if current_step % LOG_FREQUENCY == 0:
        print('Step {}, Loss_Gy {}'.format(current_step, loss_Gy.item()))
      loss_Gy.backward()  

      #Second step
      outputs = dann_net.forward(source_images, alpha) #backward pass
      domain_labels = torch.zeros(source_labels.size(0), dtype=torch.int64).to(DEVICE)
      loss_Gd_source = criterion(outputs, domain_labels)
      running_loss_Gd_source += loss_Gd_source.item()
      if current_step % LOG_FREQUENCY == 0:
        print('Step {}, Loss_Gd(source data) {}'.format(current_step, loss_Gd_source.item()))
      loss_Gd_source.backward()

      #third step
      try:
          target_images, target_lables = next(target_dataloader_iterator)
      except:
          target_dataloader_iterator = iter(target_dataloader)
          target_images, target_lables = next(target_dataloader_iterator)
      target_images = target_images.to(DEVICE)
      target_lables = target_lables.to(DEVICE)
      domain_labels = torch.ones(target_lables.size(0), dtype=torch.int64).to(DEVICE)
      outputs = dann_net.forward(target_images, alpha) #backward pass
      loss_Gd_target = criterion(outputs, domain_labels)
      running_loss_Gd_target += loss_Gd_target.item()
      if current_step % LOG_FREQUENCY == 0:
        print('Step {}, Loss_Gd(target data) {}'.format(current_step, loss_Gd_target.item()))  
      loss_Gd_target.backward()

      optimizer.step() # update weights based on accumulated gradients

      current_step += 1
    losses_Gy[j]        = running_loss_Gy        / n_loss_print
    losses_Gd_source[j] = running_loss_Gd_source / n_loss_print
    losses_Gd_target[j] = running_loss_Gd_target / n_loss_print
    accuracies_Gy[j] = test_accuracy(dann_net, source_dataloader, 'source')  # at each epoch
    j += 1

    # Step the scheduler
    scheduler.step()

 
  plt.plot(accuracies_Gy, 'r',label = 'Accuracy on Source(photo)')
  plt.legend(loc = 'best')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy values')
  plt.grid()
  plt.show()
  print_loss(losses_Gy, 'Training loss(object Classifier)')
  print_loss(losses_Gd_source, 'Training loss(Domain Discriminator photo)')
  print_loss(losses_Gd_target, 'Training loss(Domain Discriminator art painting)')
 

  test(dann_net, target_dataloader, target_dataset)

**Train with AlexNet**

In [0]:
criterion = nn.CrossEntropyLoss() 

mean_accuracy_best = 0
mean_accuracy_curr = 0
best_accuracy_train = []
best_accuracy_val_c = []
best_accuracy_val_s = []
best_loss = []
best_lr = 0 
best_batch = 0
best_step = 0
best_epoch = 0
i = 0

for i in range(len(LR)):
  #define the net
  alexnet_net = alexnet(pretrained = True) 
  alexnet_net.classifier[6] = nn.Linear(4096, NUM_CLASSES) 
  alexnet_net = alexnet_net.to(DEVICE) 

  n_epoch = NUM_EPOCHS[i]              
  learning_rate = LR[i]      
  ss = STEP_SIZE[i]                  
  bs = BATCH_SIZE[i]    

  alexnet_loss = np.empty(n_epoch)     
  j = 0 
  current_step = 0
  accuracies_train = np.empty(n_epoch)
  accuracies_val_cartoon = np.empty(n_epoch)
  accuracies_val_sketch = np.empty(n_epoch)

  train_dataloader         = DataLoader(train_dataset, batch_size = bs, shuffle = True,  num_workers = 4, drop_last = True)
  val_dataloader_cartoon   = DataLoader(val_cartoon,   batch_size = bs, shuffle = True,  num_workers = 4)
  val_dataloader_sketch    = DataLoader(val_sketch,   batch_size = bs, shuffle = True,  num_workers = 4)

  n_loss_print = len(train_dataloader)
  cudnn.benchmark
  parameters_to_optimize = alexnet_net.parameters()
  optimizer = optim.SGD(parameters_to_optimize, lr = learning_rate, momentum = MOMENTUM, weight_decay = WEIGHT_DECAY)
  scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = ss, gamma = GAMMA)

  # Start iterating over the epochs
  for epoch in range(n_epoch):
    print('Starting epoch {}/{}, LR = {}, batch size = {}'.format(epoch+1, n_epoch, scheduler.get_lr(), bs))
    running_loss = 0.0
    # Iterate over the dataset
    for images, labels in train_dataloader:
      # Bring data over the device of choice
      images = images.to(DEVICE)
      labels = labels.to(DEVICE)

      alexnet_net.train() 

      optimizer.zero_grad() 

      outputs = alexnet_net(images)

      loss = criterion(outputs, labels)
      running_loss += loss.item()

      if current_step % LOG_FREQUENCY == 0:
        print('Step {}, Loss {}'.format(current_step, loss.item()))

      # Compute gradients for each layer and update weights
      loss.backward()  # backward pass: computes gradients
      optimizer.step() # update weights based on accumulated gradients

      current_step += 1
    alexnet_loss[j] = running_loss / n_loss_print
    accuracies_train[j] = test_accuracy(alexnet_net, train_dataloader, 'train')  

    #EVALUATION ON VALIDATION
    accuracies_val_cartoon[j] = test_accuracy(alexnet_net, val_dataloader_cartoon, 'validation on cartoon')  
    accuracies_val_sketch[j] = test_accuracy(alexnet_net, val_dataloader_sketch, 'validation on sketch')  
    print('accuracies_val_sketch[{}] = {}'.format(j, accuracies_val_sketch[j]))
    print('accuracies_val_cartoon[{}] = {}'.format(j, accuracies_val_cartoon[j]))


    j += 1
    # Step the scheduler
    scheduler.step()
  #I use the last values of accuracy on cartoon and sketch for the computation of the average
  mean_accuracy_curr = (accuracies_val_cartoon[j - 1] + accuracies_val_sketch[j - 1]) /2
  print('mean accuracy curr = {} + {} / 2 = {}'.format(accuracies_val_cartoon[j - 1], accuracies_val_sketch[j - 1], mean_accuracy_curr))
  if(mean_accuracy_curr > mean_accuracy_best):
    mean_accuracy_best = mean_accuracy_curr
    best_lr = learning_rate
    best_batch = bs
    best_step = ss
    best_epoch = n_epoch 
    best_alexnet_net = copy.deepcopy(alexnet_net)
    best_accuracy_train = copy.deepcopy(accuracies_train)
    best_accuracy_val_c = copy.deepcopy(accuracies_val_cartoon)
    best_accuracy_val_s = copy.deepcopy(accuracies_val_sketch)
    best_loss = copy.deepcopy(alexnet_loss)



print('best mean accuracy: {}'.format(mean_accuracy_best))
print('best hiperparameters: ')
print('lr = {}' .format(best_lr))
print('batch size = {}' .format(best_batch))
print('step size = {}' .format(best_step))
print('epoch = {}' .format(best_epoch))

print_loss(best_loss, 'Training loss') 
print_accuracy(best_accuracy_train, best_accuracy_val_c, 'Accuracy on Train', 'Accuracy on validation(Cartoon)')  
print_accuracy(best_accuracy_train, best_accuracy_val_s, 'Accuracy on Train', 'Accuracy on validation(Sketch)')

test_dataloader = DataLoader(test_dataset,  batch_size = best_batch, shuffle = False, num_workers = 4)
test(best_alexnet_net, test_dataloader, test_dataset)

**Train with dann**

In [0]:
criterion = nn.CrossEntropyLoss() 

mean_accuracy_best = 0
mean_accuracy_curr_cartoon = 0
mean_accuracy_curr_sketch = 0
best_lr = 0 
best_batch = 0
best_alpha = 0

i = 0


for i in range(len(BATCH_SIZE_DANN)):


  bs = BATCH_SIZE_DANN[i] 
  source_dataloader        = DataLoader(source_dataset, batch_size = bs, shuffle = True,  num_workers = 4, drop_last = True)
  target_dataloader        = DataLoader(target_dataset, batch_size = bs, shuffle = False, num_workers = 4)
  val_dataloader_cartoon   = DataLoader(val_cartoon,    batch_size = bs, shuffle = True,  num_workers = 4)
  val_dataloader_sketch    = DataLoader(val_sketch,     batch_size = bs, shuffle = True,  num_workers = 4)
  validations = [val_dataloader_cartoon, val_dataloader_sketch]
  cudnn.benchmark
  
  n_loss_print = len(source_dataloader)
  k = 0

  for k in range(len(LR_DANN)): 
    learning_rate = LR_DANN[k]  


    n = 0
   
    for n in range(len(ALPHA)):
      alpha = ALPHA[n]  
      flag = 0

      accuracies_Gy_cartoon = np.empty(NUM_EPOCHS_DANN)#accuracy on object classifier on photo when the validation is cartoon   
      accuracies_Gy_sketch = np.empty(NUM_EPOCHS_DANN)#accuracy on object classifier on photo when the validation is sketch
      losses_Gy_cartoon = np.empty(NUM_EPOCHS_DANN)
      losses_Gy_sketch = np.empty(NUM_EPOCHS_DANN)

      accuracies_val_cartoon = np.empty(NUM_EPOCHS_DANN)
      accuracies_val_sketch = np.empty(NUM_EPOCHS_DANN)

      
      losses_Gd_source_cartoon = np.empty(NUM_EPOCHS_DANN) #accuracy on photo when the validation is cartoon
      losses_Gd_source_sketch = np.empty(NUM_EPOCHS_DANN) #accuracy on photo when the validation is sketch
      losses_Gd_target_cartoon = np.empty(NUM_EPOCHS_DANN)
      losses_Gd_target_sketch = np.empty(NUM_EPOCHS_DANN)
      for val_dataloader in validations:
        #define the net
        dann_net = dann(pretrained = True)
        dann_net.Gy[6] = nn.Linear(4096, NUM_CLASSES)
        dann_net.Gd[6] = nn.Linear(4096, NUM_DOMAINS)

        dann_net.Gd[1].weight.data = copy.deepcopy(dann_net.Gy[1].weight.data)
        dann_net.Gd[1].bias.data   = copy.deepcopy(dann_net.Gy[1].bias.data)

        dann_net.Gd[4].weight.data = copy.deepcopy(dann_net.Gy[4].weight.data)
        dann_net.Gd[4].bias.data   = copy.deepcopy(dann_net.Gy[4].bias.data)
        dann_net = dann_net.to(DEVICE)

        parameters_to_optimize = dann_net.parameters() 
        optimizer = optim.SGD(parameters_to_optimize, lr = learning_rate, momentum = MOMENTUM, weight_decay = WEIGHT_DECAY)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = STEP_SIZE_DANN, gamma = GAMMA)
        
        j = 0 
        current_step = 0


        # Start iterating over the epochs
        for epoch in range(NUM_EPOCHS_DANN):
          print('Starting epoch {}/{}, LR = {}, batch size = {}, alpha = {}'.format(epoch+1, NUM_EPOCHS_DANN, scheduler.get_lr(), bs, alpha))
          target_dataloader_iterator = iter(val_dataloader)
          running_loss_Gy = 0.0
          running_loss_Gd_source = 0.0
          running_loss_Gd_target = 0.0
          # Iterate over the dataset
          for source_images, source_labels in source_dataloader:
            
            source_images = source_images.to(DEVICE)
            source_labels = source_labels.to(DEVICE)

            
            dann_net.train() # Sets module in training mode
            optimizer.zero_grad() # Zero-ing the gradients

            # First step
            outputs = dann_net.forward(source_images) # forward pass
            loss_Gy = criterion(outputs, source_labels)
            running_loss_Gy += loss_Gy.item()
            if current_step % LOG_FREQUENCY == 0:
              print('Step {}, Loss_Gy {}'.format(current_step, loss_Gy.item()))
            loss_Gy.backward()  

            #Second step
            outputs = dann_net.forward(source_images, alpha) #backward pass
            domain_labels = torch.zeros(source_labels.size(0), dtype=torch.int64).to(DEVICE)
            loss_Gd_source = criterion(outputs, domain_labels)
            running_loss_Gd_source += loss_Gd_source.item()
            if current_step % LOG_FREQUENCY == 0:
              print('Step {}, Loss_Gd(source data) {}'.format(current_step, loss_Gd_source.item()))
            loss_Gd_source.backward()

            #third step
            try:
                target_images, target_lables = next(target_dataloader_iterator)
            except:
                target_dataloader_iterator = iter(val_dataloader)
                target_images, target_lables = next(target_dataloader_iterator)
            target_images = target_images.to(DEVICE)
            target_lables = target_lables.to(DEVICE)
            domain_labels = torch.ones(target_lables.size(0), dtype=torch.int64).to(DEVICE)
            outputs = dann_net.forward(target_images, alpha) #backward pass
            loss_Gd_target = criterion(outputs, domain_labels)
            running_loss_Gd_target += loss_Gd_target.item()
            if current_step % LOG_FREQUENCY == 0:
              print('Step {}, Loss_Gd(target data) {}'.format(current_step, loss_Gd_target.item()))  
            loss_Gd_target.backward()

            optimizer.step() 

            current_step += 1
          if(flag == 0):
            losses_Gy_cartoon[j] = running_loss_Gy / n_loss_print # loss on classifier on photo when validation is cartoon
            accuracies_Gy_cartoon[j] = test_accuracy(dann_net, source_dataloader, 'source')  # accuracy on classifier on photo when validation is cartoon
            losses_Gd_source_cartoon[j] = running_loss_Gd_source / n_loss_print #loss on domain discriminator(source) when the validation is cartoon
            losses_Gd_target_cartoon[j] = running_loss_Gd_target / n_loss_print #loss on domain discriminator(target) when the validation is cartoon
            accuracies_val_cartoon[j] = test_accuracy(dann_net, val_dataloader_cartoon, 'validation on cartoon')  # accuracy on domain discriminator when the validation in cartoon
            print('accuracies_val_cartoon[{}] = {}'.format(j, accuracies_val_cartoon[j]))
            
          else:
            losses_Gy_sketch[j] = running_loss_Gy / n_loss_print # loss on classifier on photo when validation is sketch
            accuracies_Gy_sketch[j] = test_accuracy(dann_net, source_dataloader, 'source')  # accuracy on classifier on photo when validation is sketch
            losses_Gd_source_sketch[j] = running_loss_Gd_source / n_loss_print #loss on domain discriminator when the validation is sketch
            losses_Gd_target_sketch[j] = running_loss_Gd_target / n_loss_print #loss on domain discriminator(target) when the validation is sketch
            accuracies_val_sketch[j] = test_accuracy(dann_net, val_dataloader_sketch, 'validation on sketch')  # at each epoch
            print('accuracies_val_sketch[{}] = {}'.format(j, accuracies_val_sketch[j]))
          j += 1

          # Step the scheduler
          scheduler.step()
        flag = 1 
      mean_accuracy_curr = (accuracies_val_cartoon[j - 1] + accuracies_val_sketch[j - 1]) / 2
      print('value of j = {}'.format(j))
      print('mean accuracy curr = {} + {} / 2 = {}'.format(accuracies_val_cartoon[j - 1], accuracies_val_sketch[j - 1], mean_accuracy_curr))
      if(mean_accuracy_curr > mean_accuracy_best):
        mean_accuracy_best = mean_accuracy_curr
        best_lr = learning_rate
        best_batch = bs
        best_alpha = alpha
        #values of cartoon
        best_loss_Gy_c_dann = copy.deepcopy(losses_Gy_cartoon)
        best_acc_Gy_c_dann = copy.deepcopy(accuracies_Gy_cartoon)
        best_loss_Gd_s_c_dann = copy.deepcopy(losses_Gd_source_cartoon)
        best_loss_Gd_t_c_dann = copy.deepcopy(losses_Gd_target_cartoon)
        best_acc_val_c_dann = copy.deepcopy(accuracies_val_cartoon)
        #values of sketch
        best_loss_Gy_sk__dann = copy.deepcopy(losses_Gy_sketch)
        best_acc_Gy_sk_dann = copy.deepcopy(accuracies_Gy_sketch)
        best_loss_Gd_s_sk_dann = copy.deepcopy(losses_Gd_source_sketch)
        best_loss_Gd_t_sk_dann = copy.deepcopy(losses_Gd_target_sketch)
        best_acc_val_sk_dann = copy.deepcopy(accuracies_val_sketch)
      
print('best mean accuracy: {}'.format(mean_accuracy_best))
print('best hiperparameters: ')
print('lr = {}' .format(best_lr))
print('batch size = {}' .format(best_batch))
print('step size = {}' .format(STEP_SIZE_DANN))
print('epoch = {}' .format(NUM_EPOCHS_DANN))
print('alpha = {}' .format(best_alpha))
#print of loss and accuracy of the best net (validation)
print_loss(best_loss_Gy_c_dann, 'Source loss Gy(cartoon)') 
print_loss(best_loss_Gy_sk__dann, 'Source loss Gy(sketch)') 

print_loss(best_loss_Gd_s_c_dann, 'Source loss Gd(cartoon)') 
print_loss(best_loss_Gd_s_sk_dann, 'Source loss Gd(sketch)')

print_loss(best_loss_Gd_t_c_dann, 'Target loss Gd(cartoon)') 
print_loss(best_loss_Gd_t_sk_dann, 'Target loss Gd(sketch)') 

print_accuracy(best_acc_Gy_c_dann, best_acc_val_c_dann, 'Accuracy on Source(photo)', 'Accuracy on Validation(Cartoon)')  
print_accuracy(best_acc_Gy_sk_dann, best_acc_val_sk_dann, 'Accuracy on Source(photo)', 'Accuracy on Validation(Sketch)')


**Evaluation of DANN**

In [0]:
        best_lr = 0.005 
        best_batch = 128
        best_alpha = 0.1
        final_train_and_test_dann(best_lr, best_batch, STEP_SIZE_DANN, NUM_EPOCHS_DANN, best_alpha)